In [2]:
!pip install PyQt5

  Obtaining dependency information for PyQt5 from https://files.pythonhosted.org/packages/ca/ac/596e8ca16fd0634542d874c0d79219fc527ea7de73a5000092f60ecbf6e9/PyQt5-5.15.10-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for PyQt5-sip<13,>=12.13 from https://files.pythonhosted.org/packages/8e/e2/d296cb17bae19ba49137a2649934a70d5f48cc99b7daa6ce9cf1aecddfeb/PyQt5_sip-12.13.0-cp311-cp311-win_amd64.whl.metadata
  Using cached PyQt5_Qt5-5.15.2-py3-none-win_amd64.whl (50.1 MB)
Using cached PyQt5-5.15.10-cp37-abi3-win_amd64.whl (6.8 MB)
Using cached PyQt5_sip-12.13.0-cp311-cp311-win_amd64.whl (78 kB)



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib qt
import seaborn as sns
import os
import datetime
import xlrd
plt.style.use('seaborn-v0_8-ticks')
from scipy.signal import find_peaks
import itertools
from pylab import *

Pobranie danych

In [21]:
folder_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\controlled_breathing"
folders = os.listdir(folder_path)
breathing_dict = {folder : {} for folder in folders}

for folder_name in folders:
    path = os.path.join(folder_path, folder_name)
    for file_name in os.listdir(path):
        if file_name.endswith('.csv'):
         df_name = file_name.split('_ODD')[0]
         df = pd.read_csv(os.path.join(path, file_name))
         breathing_dict[folder_name][df_name] = df

Funkcja konwertująca czas

In [36]:
def icmp_dateformat_to_datetime(icmp_time_mark):
    datetime_date = xlrd.xldate_as_datetime(icmp_time_mark, 0)
    datetime_date = datetime_date + datetime.timedelta(hours=1)
    return datetime_date

Dodanie kolumny z datą i godziną do każdego dataframe i wypełnienie braków danych (NaN) wartościami średnimi

In [ ]:
for breath_freq in breathing_dict:
    for df_name, df in breathing_dict[breath_freq].items():
               df['DateFormat'] = df['DateTime'].map(icmp_dateformat_to_datetime)
               df.fillna(df.mean(), inplace=True)

Funkcja znajdująca piki, diff to jest RR w przypadku ECG tylko trzeba je przemnożyć przez 0.005 (1/(200 Hz)), peaks to numery próbek odpowiadających pikom, signal[peaks] to wartości sygnału odpowiadające tym numerom próbek

In [190]:
def get_peaks(signal):
    dist = 115
    peaks, _ = find_peaks(signal, distance=dist)
    difference = np.diff(peaks)  
    return peaks, signal[peaks], difference

Wyrysowanie 600 pierwszych próbek każdego sygnału z zaznaczonymi znalezionymi pikami ("x")

In [79]:
folder_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\timeseries_plots"
for breath_freq in breathing_dict:
    for df_name, df in breathing_dict[breath_freq].items():       
        fig, ax = plt.subplots(2, 1, tight_layout = True, figsize=(16, 20)) 
        for i, col in enumerate(['abp_arm[abp_arm]', 'ekg[ekg]']):
           x = df[col].iloc[:600]
           ax[i].plot(x)
           my_peaks = get_peaks(x)
           ax[i].plot(my_peaks[0], my_peaks[1], "x")
           ax[i].set_ylabel(f'{col}', fontsize=25)
           ax[i].tick_params(axis='both', labelsize=18)
           ax[i].grid(True)
        ax[1].set_xlabel("nr probki", fontsize = 28)
        path = folder_path+f"\\{breath_freq}\\{df_name}_plot.pdf"
        plt.savefig(path, bbox_inches='tight', dpi=600)
        plt.show()
        plt.close()

In [191]:
def make_empty_matrix():
    my_permutations = [''.join(map(str, item)) for item in list((itertools.product([1, 0], repeat=3)))]
    my_matrix = pd.DataFrame()
    for col1 in my_permutations:
       for col2 in my_permutations:
            my_matrix.at[col1, col2] = 0
    return my_matrix

In [192]:
def get_diff_df(dataframe):
    sap = get_peaks(dataframe['abp_arm[abp_arm]'])[1]
    rr = get_peaks(dataframe['ekg[ekg]'])[2]*0.005
    sap_diff = np.diff(sap)
    rr_diff = np.diff(rr)
    diff_dataframe = pd.DataFrame(list(zip(sap_diff, rr_diff)), columns=['sap_diff', 'rr_diff'])
    return diff_dataframe

In [193]:
def make_and_save_heatmap(normalised_array, breath_frequency):
    my_custom_palette = sns.color_palette("ch:-.24", as_cmap=True)
    heatmap_path = f"C:\\Users\\48503\\Desktop\\joint_symbolic\\heatmaps\\{breath_frequency}_heatmap.pdf"
    my_heatmap = sns.heatmap(normalised_array,cmap = my_custom_palette, annot=True)
    my_heatmap.xaxis.tick_top()
    my_heatmap.xaxis.set_ticks_position('none')
    fontsize = 10
    plt.xlabel('SAP', fontsize = fontsize) 
    my_heatmap.xaxis.set_label_position('top') 
    plt.ylabel('RR', fontsize = fontsize) 
    plt.savefig(heatmap_path, bbox_inches='tight', dpi=600)
    plt.show()
    plt.close()

In [199]:
def joint_symbolic(data_dict):
    for freq in data_dict:
        freq_matrix = make_empty_matrix()
        for name, dataframe in data_dict[freq].items():
            diff_df = get_diff_df(dataframe)
            
            for col_name in ['sap_diff', 'rr_diff']:
                diff_df[f'{col_name}_binary'] = diff_df.apply(lambda row: 1 if row[col_name] > 0 else 0, axis = 1)
            
            for index in range(0, len(diff_df)):
                binary_sap =  ''.join(diff_df['sap_diff_binary'].iloc[index:index+3].astype(str))
                binary_rr =  ''.join(diff_df['rr_diff_binary'].iloc[index:index+3].astype(str))
                if len(binary_rr) > 2 and len(binary_sap) > 2:
                     freq_matrix.at[binary_rr, binary_sap] += 1
                
        max_value = max(freq_matrix.max(axis=1))        
        normalised_matrix = freq_matrix.div(max_value)        
        make_and_save_heatmap(normalised_matrix, freq)

In [200]:
joint_symbolic(breathing_dict)